In [1]:
import uproot # python3 -m pip install uproot 
import numpy as np
file = uproot.open("DATA/Run1_4DS_ana.root") # load the ROOT file 
hist=file["TrackSummary/FitResults/Times_gt_1800MeV"] # load histogram
hist_data = hist.numpy() # the histogram in the numpy format
counts, edges = hist_data[0], hist_data[1] # len(edges) = len(counts) + 1 

In [2]:
#set some constants and filter data 
N=int(np.sum(counts))
print(N, "enties in", len(counts), "bins before filtering")

t_mod = 100 # us; fold plot every 100 us
binW=edges[1] - edges[0] # us bin wdith (~150ns)
i_t_mod=int(t_mod/binW) # the index at 100 us 

#remove data before scraping
t_min=30 # us ;
edges_filter=edges[edges>=t_min] #remove all times less than t_min
ind_fil=np.nonzero(edges>=t_min)[0][0] # find the first index 
counts_filter=counts[ind_fil:] # slice for counts with the index
N=int(np.sum(counts_filter))
print(N, "enties in", len(counts), "bins after filtering with t>=",t_min, "us")

101968592 enties in 6000 bins before filtering
60195844 enties in 6000 bins after filtering with t>= 30 us


In [3]:
import matplotlib.pyplot as plt
x=edges_filter
y=np.append(counts_filter, 0)
plt.step(x=x, y=y, where="post", color="g")

plt.xlim(0, t_mod*2) 

#log 
plt.ylim(0, y.max() * 1.1)


plt.ylabel("Counts per "+str(int(binW*1e3))+" ns", fontsize=18)
plt.xlabel(r"Time [$\mathrm{\mu}$s] modulo "+str(t_mod)+" $\mathrm{\mu}$s", fontsize=18);
plt.tight_layout()
plt.savefig("fig/wiggle.png", dpi=300)

In [4]:
import sys, os, math
sys.path.append("Blinding/") # folder with Blinders and libBlinders.so
from BlindersPy3 import Blinders
from BlindersPy3 import FitType
getBlinded = Blinders(FitType.Omega_a, "EDM all day") # will tell you that you are blinded

Define blinded fit function $N(t)=Ne^{-t/\tau}[1+A\cos(\omega_at+\phi)]$,
where  
[0] $N$ is the overall normalisation  
[1] $\tau$ is the boosted muon lifetime $\tau = \gamma \cdot \tau_0 = 29.3\cdot2.2=66.44 \, \mu$s  
[2] $A$ is the asymmetry  
[3] $\omega_a$ is the anomalous precision frequency (blinded)  
[4] $\phi$ is the initial phase  

In [5]:
def blinded_wiggle_function(x, *pars):
    norm  = pars[0]
    life  = pars[1]
    asym  = pars[2]
    R     = pars[3]
    phi   = pars[4]
    
    time  = x[0]
    omega = getBlinded.paramToFreq(R)
    
    return norm * math.exp(-time/life) * (1 + asym*math.cos(omega*time + phi))

In [6]:
from scipy import optimize, stats

y_err = np.sqrt(y)
# p0=(2.0e+05, 64.4, 0.5, 0.0, 0.0) # initial guess of the fit parameters

p0=(1, 1, 1, 1, 1)

#function, X, Y, [starting par]
# Levenberg-Marquardt algorithm as implemented in MINPACK

# add y-errors 

par, pcov = optimize.curve_fit(f=blinded_wiggle_function, xdata=x, ydata=y, p0=p0, absolute_sigma=False, method='lm')
par_e = np.sqrt(np.diag(pcov))

print("Params:", par)
print("Errors:", par_e)
print("Cov:", pcov)

Params: [1. 1. 1. 1. 1.]
Errors: [           nan 1.94881770e+21 0.00000000e+00 0.00000000e+00
 2.08696847e+24]
Cov: [[-7.05505610e+46  1.36171790e+45 -1.10217526e+47 -1.73606311e+52
   2.38389173e+47]
 [-4.25335683e+45  3.79789044e+42  8.50671366e+45  0.00000000e+00
  -0.00000000e+00]
 [ 0.00000000e+00  2.42159285e+45  0.00000000e+00  3.56797431e+52
  -1.08885935e+48]
 [-5.35196147e+52 -1.52680029e+50  7.13594862e+52  0.00000000e+00
  -1.42718972e+53]
 [ 1.08885935e+48  2.27557603e+46 -2.17771870e+48  0.00000000e+00
   4.35543739e+48]]


/Users/gleb/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sqrt
  
